In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sys
sys.path.append('../Task 1/')
from efficient_apriori import apriori
from improved_apriori import Improved_Apriori
import json
import time
import os
import itertools
import ast
from tqdm import tqdm
import collections
import math
import random
import ijson

In [16]:

# Specify the paths for your JSON and CSV files
json_file = 'dataset/yelp_academic_dataset_review.json'
csv_file = 'dataset/yelp_academic_dataset_review.csv'

size = 1000000
reviews = pd.read_json(json_file, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

chunk_list = []
for chunk_review in reviews:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)

    chunk_list.append(chunk_review)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
# Save the DataFrame to a CSV file
df.to_csv(csv_file, index=False)

print(f'Conversion completed. CSV file saved as {csv_file}')

Conversion completed. CSV file saved as dataset/yelp_academic_dataset_review.csv


In [18]:
# Process the dataset by chunks into username and the list of yelp the user watched
def process_yelp_chunk(df, carry_over):
    if carry_over is not None:
        df = pd.concat([carry_over, df])
    groups = df.groupby('user_id')['business_id'].apply(list)
    last_user = df.iloc[-1]['user_id']
    if last_user in groups:
        carry_over = df[df['user_id'] == last_user]
        groups = groups.drop(last_user)
    else:
        carry_over = None
    return groups, carry_over

In [19]:
carry_over = None
chunksize = 100000 # adjust this value depending on your available memory
# Might have to figure out a way to shuffle the dataset 
if(not os.path.exists('dataset/processed_yelp_output.txt')):
    with open('dataset/processed_yelp_output.txt', 'w') as f:
        for chunk in pd.read_csv(csv_file_name, chunksize=chunksize):
            groups, carry_over = process_yelp_chunk(chunk, carry_over)
            for user, yelp_list in groups.items():
                f.write(f'{user} {yelp_list}\n')

        # don't forget to process the last carry_over
        if carry_over is not None:
            groups, _ = process_yelp_chunk(carry_over, None)
            for user, yelp_list in groups.items():
                f.write(f'{user} {yelp_list}\n')

In [20]:
# Shuffle the text in chunks 
def shuffle_large_file(file_name, output_file_name, chunk_size):
    with open(file_name, 'r') as f:
        while True:
            lines = list(itertools.islice(f, chunk_size))
            if not lines:
                break
            random.shuffle(lines)
            with open(output_file_name, 'a') as out:
                out.write(''.join(lines))


# Call the function with your parameters
if(not os.path.exists('dataset/processed_yelp_output_shuffled.txt')):
    shuffle_large_file('dataset/processed_yelp_output.txt', 'dataset/processed_yelp_output_shuffled.txt', 3000000)

In [21]:
def read_file_in_partitions(file_path, partition_size):
    with open(file_path, 'r') as file:
        partition = []
        for line in file:
            partition.append(line)
            if len(partition) >= partition_size:
                yield partition
                partition = []
        if partition:  # yield any remaining lines
            yield partition

In [22]:
# Global variable to get the counts of all itemsets
global_counts = {}
def generate_global_counts(partition, global_candidates):

    # For 1th itemset, generate the transaction id list for the ith partition 
    transaction_id_dict = collections.defaultdict(list)
    for transaction_id in partition:
        for item in partition[transaction_id]:
            item_tuple = (item,)
            transaction_id_dict[item_tuple].append(transaction_id)

    # Filter based on the global candidates formed
    transaction_ids_dict = {item: transaction_ids for item, transaction_ids in transaction_id_dict.items() if item in global_candidates[1]}

    # Get the global count of all 1th itemset
    for item in transaction_id_dict:
        if(len(item) not in global_counts):
            global_counts[len(item)] = {}
        if(item not in global_counts[len(item)]):
            global_counts[len(item)][item] = len(transaction_id_dict[item])
        else:
            global_counts[len(item)][item] += len(transaction_id_dict[item])

    # Extend to find global count of all nth itemset from the global candidates
    for i in tqdm(range(1, len(global_candidates))):
        for itemset in global_candidates[i+1]:
            transaction_ids = set(transaction_id_dict[(itemset[0],)])
            for i in range(1, len(itemset)):
                # We are only interested in the transactions where all items in itemset is present
                transaction_ids = transaction_ids.intersection(set(transaction_ids_dict.get((itemset[i],), {})))
            if(len(itemset) not in global_counts):
                global_counts[len(itemset)] = {}

            if(itemset not in global_counts[len(itemset)]):
                global_counts[len(itemset)][itemset] = len(transaction_ids)
            else:
                global_counts[len(itemset)][itemset] += len(transaction_ids)



In [37]:
file_path = 'dataset/processed_yelp_output_shuffled.txt'
size_of_data = sum(1 for line in open(file_path))
min_support=0.001
partition_size = 5000
num_partitions = size_of_data// partition_size
print(num_partitions)
partition_candidates = []
global_candidates = collections.defaultdict(list)
# Step 1: Partitioning
global_min_support = math.ceil((min_support*size_of_data)/num_partitions)
for i, partition in enumerate(read_file_in_partitions(file_path, partition_size)):
    print(f'Partition {i+1}:')
    dict_yelp = {}
    for line in partition:
        user, yelp_list_str = line.strip().split(' ', 1)
        yelp_list = ast.literal_eval(yelp_list_str)
        dict_yelp[user] = yelp_list
   
    improved_apriori = Improved_Apriori(dict_yelp, min_support=min_support, min_confidence=1, verbose=2)
    # Step 2: Retreieve frequent itemset per partition
    partition_frequent_itemset = improved_apriori.apriori()
    # Efficient Apriori for sanity check
    # partition_frequent_itemset, _ = apriori(list(dict_yelp.values()), min_support = min_support, verbosity=2)

    # Form the global candidate set from the large itemset in each partitions
    # In this space, we ignore the count of itemset in each partition as they are not useful in our global support count
    # All they do is just show the itemset was large enough in the current partition
    # Merging Phase
    for level, itemset in partition_frequent_itemset.items():
        for key in itemset.keys():
            if(key not in global_candidates[level]):
                global_candidates[level].append(key)

1121
Partition 1:
Found 36 frequent itemsets from 1th item candidate sets
Found 630 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 0.0


100%|█████████████████████████████████████████████████████████████████████████████| 630/630 [00:00<00:00, 88478.54it/s]


Found 0 frequent itemsets from 2th item candidate sets
Partition 2:
Found 41 frequent itemsets from 1th item candidate sets
Found 820 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 0.0


100%|████████████████████████████████████████████████████████████████████████████| 820/820 [00:00<00:00, 408762.69it/s]


Found 0 frequent itemsets from 2th item candidate sets
Partition 3:
Found 41 frequent itemsets from 1th item candidate sets
Found 820 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 0.0


...

Found 57 frequent itemsets from 1th item candidate sets
Found 1596 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 0.00010609626770019531


100%|██████████████████████████████████████████████████████████████████████████| 1596/1596 [00:00<00:00, 390304.31it/s]


Found 0 frequent itemsets from 2th item candidate sets
Partition 1122:
Found 53 frequent itemsets from 1th item candidate sets
Found 1378 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 0.0


100%|██████████████████████████████████████████████████████████████████████████| 1378/1378 [00:00<00:00, 346736.51it/s]

Found 0 frequent itemsets from 2th item candidate sets


In [38]:
# Now we have to read the lines in chunks for our disk-based operations
min_support_count = min_support * size_of_data
for i, partition in enumerate(read_file_in_partitions(file_path, partition_size)):
    print(f'Partition {i+1}:')

    # Hold the partition data in main memory
    dict_yelp = {}
    for line in partition:
        user, yelp_list_str = line.strip().split(' ', 1)
        yelp_list = ast.literal_eval(yelp_list_str)
        dict_yelp[user] = yelp_list
    # Pure disk based implementation would probably require us to save the global candidates in disk 
    generate_global_counts(dict_yelp, global_candidates)
    
        

Partition 1:


0it [00:00, ?it/s]


Partition 2:


0it [00:00, ?it/s]


Partition 3:


...

0it [00:00, ?it/s]


Partition 1121:


0it [00:00, ?it/s]


Partition 1122:


0it [00:00, ?it/s]


In [39]:
global_frequent_itemsets = {level: {itemset: count for itemset, count in itemsets.items() if count >= min_support_count} for level, itemsets in global_counts.items()}
global_frequent_itemsets

{1: {('qb28j-FNX1_6xm7u372TZA',): 7924,
  ('j-qtdD55OLfSqfsWuQTDJg',): 5718,
  ('ctHjyadbDQAtUFfkcAFEHw',): 6308,
  ('GBTPC53ZrG1ZBY3DT8Mbcw',): 9276,
  ('_C7QiQQc47AOEv4PE3Kong',): 9916,
  ('VaO-VW3e1kARkU9bP1E7Fw',): 8046,
  ('VVH6k9-ycttH3TV_lk5WfQ',): 7246,
  ('gTC8IQ_i8zXytWSly3Ttvg',): 7788,
  ('DcBLYSvOuWcNReolRVr12A',): 6406,
  ('XnQ84ylyAZwh-XfHGGNBbQ',): 6402,
  ('6ajnOk0GcY9xbb5Ocaw8Gw',): 5902,
  ('VQcCL9PiNL_wkGf-uF3fjg',): 10272,
  ('IkY2ticzHEn4QFn8hQLSWg',): 6832,
  ('C9K3579SJgLPp0oAOM29wg',): 5800,
  ('U3grYFIeu6RgAAQgdriHww',): 5990,
  ('GXFMD0Z4jEVZBCsbPf4CTQ',): 12288,
  ('dsfRniRgfbDjC8os848B6A',): 6026,
  ('_ab50qdWOk0DdB6XOrBitw',): 15308,
  ('QHWYlmVbLC3K6eglWoHVvA',): 6740,
  ('6a4gLLFSgr-Q6CZXDLzBGQ',): 8932,
  ('PP3BBaVxZLcJU54uP_wL6Q',): 8534,
  ('ac1AeYqs8Z4_e2X5M3if2A',): 14992,
  ('9PZxjhTIU7OgPIzuGi89Ew',): 6484,
  ('oBNrLz4EDhiscSlbOl8uAw',): 10498,
  ('1b5mnK8bMnnju_cvU65GqQ',): 8476,
  ('L5LLN0RafiV1Z9cddzvuCw',): 6318,
  ('ytynqOUb3hjKeJfRj5Tshw',):

In [40]:
for level in global_frequent_itemsets:
    print(len(global_frequent_itemsets[level]))

33


In [42]:
data = pd.read_csv('dataset/yelp_academic_dataset_review.csv')
data = data[['user_id', 'business_id']]
grouped_data = data.groupby('user_id')['business_id'].apply(list)
grouped_data = grouped_data.to_dict()

In [43]:
improved_apriori = Improved_Apriori(grouped_data, min_support=min_support, min_confidence=1, verbose=2)
frequent_yelp_set = improved_apriori.apriori()

Found 81 frequent itemsets from 1th item candidate sets
Found 3240 candidates for 2th item candidate sets
Time taken to find 2th item candidate sets: 0.0


100%|████████████████████████████████████████████████████████████████████████████| 3240/3240 [00:01<00:00, 2670.52it/s]

Found 0 frequent itemsets from 2th item candidate sets


In [44]:
itemset, _ = apriori(list(grouped_data.values()), min_support = min_support, verbosity=2)

Generating itemsets.
 Counting itemsets of length 1.
  Found 150346 candidate itemsets of length 1.
  Found 78 large itemsets of length 1.
    [('3YqUe2FTCQr0pPVK8oCv6Q',), ('L5LLN0RafiV1Z9cddzvuCw',), ('QHWYlmVbLC3K6eglWoHVvA',), ('GBTPC53ZrG1ZBY3DT8Mbcw',), ('U3grYFIeu6RgAAQgdriHww',), ('VaO-VW3e1kARkU9bP1E7Fw',), ('gTC8IQ_i8zXytWSly3Ttvg',), ('ReV4Q3rEJ8neicQPc6pC0w',), ('qQO7ErS_RAN4Vs1uX0L55Q',), ('_C7QiQQc47AOEv4PE3Kong',), ('VQcCL9PiNL_wkGf-uF3fjg',), ('qb28j-FNX1_6xm7u372TZA',), ('VVH6k9-ycttH3TV_lk5WfQ',), ('PP3BBaVxZLcJU54uP_wL6Q',), ('6a4gLLFSgr-Q6CZXDLzBGQ',), ('sTPueJEwcRDj7ZJmG7okYA',), ('DXDoh8MNrqjtXsZnkuk4hA',), ('ChlcxTEoWBQJXJ2Xb2vm5g',), ('VAeEXLbEcI9Emt9KGYq9aA',), ('1b5mnK8bMnnju_cvU65GqQ',), ('DcBLYSvOuWcNReolRVr12A',), ('oBNrLz4EDhiscSlbOl8uAw',), ('wz8ZPfySQczcPgSyd33-HQ',), ('TcNZXteosegb1RO4O5hREw',), ('SZU9c8V2GuREDN5KgyHFJw',), ('6ajnOk0GcY9xbb5Ocaw8Gw',), ('IkY2ticzHEn4QFn8hQLSWg',), ('_ab50qdWOk0DdB6XOrBitw',), ('GXFMD0Z4jEVZBCsbPf4CTQ',), ('9PZxjhTIU7OgP

In [45]:
frequent_yelp_set

{1: {('-QI8Qi8XWH3D8y8ethnajA',): 2235,
  ('0RuvlgTnKFbX3IK0ZOOocA',): 2733,
  ('1b5mnK8bMnnju_cvU65GqQ',): 4247,
  ('2BMk_drsikKWslJCXmQtjQ',): 2023,
  ('3YqUe2FTCQr0pPVK8oCv6Q',): 2916,
  ('3iUCCf1FWmjlFbGYvBgf9w',): 2240,
  ('6Pu5MKIWbn6FO-oebSSrYQ',): 2339,
  ('6Ty-KKWq6hLZYW8DWEHYvg',): 2396,
  ('6a4gLLFSgr-Q6CZXDLzBGQ',): 4480,
  ('6ajnOk0GcY9xbb5Ocaw8Gw',): 2974,
  ('75FY8ZQx5nOWP0VFmNvWfw',): 2083,
  ('8pqdJjefYq-a9IBSJJmKwA',): 2577,
  ('8uF-bhJFgT4Tn6DTb27viA',): 2110,
  ('9PZxjhTIU7OgPIzuGi89Ew',): 3264,
  ('9xdXS7jtWjCVzL4_oPGv9A',): 2502,
  ('AGlh4ZDv6jnoiYfz7At9mw',): 2778,
  ('BjeHLwKOlHyV6DJgmZxAjA',): 2178,
  ('C9K3579SJgLPp0oAOM29wg',): 2909,
  ('ChlcxTEoWBQJXJ2Xb2vm5g',): 2450,
  ('DXDoh8MNrqjtXsZnkuk4hA',): 2335,
  ('DcBLYSvOuWcNReolRVr12A',): 3217,
  ('EagkHaaC-kUozD3MPzbRIw',): 2440,
  ('GBTPC53ZrG1ZBY3DT8Mbcw',): 4661,
  ('GXFMD0Z4jEVZBCsbPf4CTQ',): 6160,
  ('IWHdx0NhDKADkGOgXgOFKQ',): 2245,
  ('I_3LMZ_1m2mzR0oLIOePIg',): 4093,
  ('IkY2ticzHEn4QFn8hQLSWg',): 3428

In [46]:
itemset

{1: {('3YqUe2FTCQr0pPVK8oCv6Q',): 2860,
  ('L5LLN0RafiV1Z9cddzvuCw',): 3066,
  ('QHWYlmVbLC3K6eglWoHVvA',): 3261,
  ('GBTPC53ZrG1ZBY3DT8Mbcw',): 4554,
  ('U3grYFIeu6RgAAQgdriHww',): 2940,
  ('VaO-VW3e1kARkU9bP1E7Fw',): 3966,
  ('gTC8IQ_i8zXytWSly3Ttvg',): 3837,
  ('ReV4Q3rEJ8neicQPc6pC0w',): 2035,
  ('qQO7ErS_RAN4Vs1uX0L55Q',): 2062,
  ('_C7QiQQc47AOEv4PE3Kong',): 4877,
  ('VQcCL9PiNL_wkGf-uF3fjg',): 5070,
  ('qb28j-FNX1_6xm7u372TZA',): 3902,
  ('VVH6k9-ycttH3TV_lk5WfQ',): 3582,
  ('PP3BBaVxZLcJU54uP_wL6Q',): 4250,
  ('6a4gLLFSgr-Q6CZXDLzBGQ',): 4422,
  ('sTPueJEwcRDj7ZJmG7okYA',): 2736,
  ('DXDoh8MNrqjtXsZnkuk4hA',): 2301,
  ('ChlcxTEoWBQJXJ2Xb2vm5g',): 2417,
  ('VAeEXLbEcI9Emt9KGYq9aA',): 3054,
  ('1b5mnK8bMnnju_cvU65GqQ',): 4207,
  ('DcBLYSvOuWcNReolRVr12A',): 3160,
  ('oBNrLz4EDhiscSlbOl8uAw',): 5193,
  ('wz8ZPfySQczcPgSyd33-HQ',): 3346,
  ('TcNZXteosegb1RO4O5hREw',): 2160,
  ('SZU9c8V2GuREDN5KgyHFJw',): 2404,
  ('6ajnOk0GcY9xbb5Ocaw8Gw',): 2893,
  ('IkY2ticzHEn4QFn8hQLSWg',): 3401